Having a look at the data

In [1]:
# import packages

import gc
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format # disabling scientific notation in pandas
import ydata_profiling 
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload
%autoreload 2

In [2]:
#train_df_cat = pd.read_csv("data/train_categorial_features_moda.csv")
#train_df_cat['fullVisitorId'] = train_df_cat['fullVisitorId'].astype(str) 

/tmp/ipykernel_1878/1819675655.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df_cat = pd.read_csv("data/train_categorial_features_moda.csv")


In [3]:
#train_df_cat.shape

(714167, 14)

In [4]:
#train_df_cat.head()

,fullVisitorId,device_browser,device_operatingSystem,geoNetwork_city,geoNetwork_metro,geoNetwork_country,geoNetwork_networkDomain,geoNetwork_region,trafficSource_adContent,trafficSource_campaign,trafficSource_keyword,trafficSource_source,trafficSource_referralPath,weekday
0,10278554503158,0.00,1.00,1.00,0.00,68.00,397.00,1.00,0.00,0.00,0.00,0.00,0.00,3.00
1,20424342248747,0.00,0.00,45.00,0.00,42.00,"6,726.00",37.00,0.00,0.00,0.00,0.00,0.00,2.00
2,27376579751715,0.00,1.00,1.00,0.00,33.00,90.00,1.00,0.00,0.00,0.00,0.00,0.00,5.00
3,39460501403861,0.00,0.00,1.00,0.00,11.00,133.00,1.00,0.00,0.00,0.00,25.00,72.00,0.00
4,40862739425590,0.00,1.00,110.00,2.00,33.00,109.00,21.00,0.00,1.00,13.00,0.00,0.00,5.00


In [5]:
#train_df_cat.dtypes

fullVisitorId                  object
device_browser                float64
device_operatingSystem        float64
geoNetwork_city               float64
geoNetwork_metro              float64
geoNetwork_country            float64
geoNetwork_networkDomain      float64
geoNetwork_region             float64
trafficSource_adContent       float64
trafficSource_campaign        float64
trafficSource_keyword         float64
trafficSource_source          float64
trafficSource_referralPath    float64
weekday                       float64
dtype: object

In [6]:
train_df_filter = pd.read_csv("data/train_filtered.csv")
train_df_filter['fullVisitorId'] = train_df_filter['fullVisitorId'].astype(str)

/tmp/ipykernel_1878/537917179.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df_filter = pd.read_csv("data/train_filtered.csv")


In [7]:
train_df_filter.shape

(902755, 32)

In [8]:
train_df_filter.head()

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_transactionRevenue,...,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20160902,1131660440785968503,1,1472830385,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
1,Organic Search,20160902,377306020877927890,1,1472880147,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
2,Organic Search,20160902,3895546263509774583,1,1472865386,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
3,Organic Search,20160902,4763447161404445595,1,1472881213,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,google + online,organic,@,google
4,Organic Search,20160902,27294437909732085,2,1472822600,1.00,1,0.00,1.00,0.00,...,@,1,0.00,@,@,1,@,organic,@,google


In [9]:
train_df_filter.dtypes

channelGrouping                                  object
date                                              int64
fullVisitorId                                    object
visitNumber                                       int64
visitStartTime                                    int64
totals_bounces                                  float64
totals_hits                                       int64
totals_newVisits                                float64
totals_pageviews                                float64
totals_transactionRevenue                       float64
device_browser                                   object
device_deviceCategory                            object
device_isMobile                                   int64
device_operatingSystem                           object
geoNetwork_city                                  object
geoNetwork_continent                             object
geoNetwork_country                               object
geoNetwork_metro                                

In [10]:
#rain_df_flat = pd.read_csv("data/train_flat.csv")
#train_df_flat['fullVisitorId'] = train_df_flat['fullVisitorId'].astype(str)

/tmp/ipykernel_1878/3077697282.py:1: DtypeWarning: Columns (2,49) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df_flat = pd.read_csv("data/train_flat.csv")


In [11]:
#train_df_flat.shape

(903653, 55)

In [12]:
#train_df_flat.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,totals_bounces,totals_hits,...,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,0,1472830385,1,1472830385,1.00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,0,1472880147,1,1472880147,1.00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,0,1472865386,1,1472865386,1.00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,0,1472881213,1,1472881213,1.00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,0,1472822600,2,1472822600,1.00,1,...,NaN,NaN,NaN,NaN,NaN,True,NaN,organic,NaN,google


In [13]:
#train_df_flat.dtypes

channelGrouping                                       object
date                                                   int64
fullVisitorId                                         object
sessionId                                             object
socialEngagementType                                   int64
visitId                                                int64
visitNumber                                            int64
visitStartTime                                         int64
totals_bounces                                       float64
totals_hits                                            int64
totals_newVisits                                     float64
totals_pageviews                                     float64
totals_transactionRevenue                            float64
totals_visits                                          int64
device_browser                                        object
device_browserSize                                   float64
device_browserVersion   

In [14]:
#train_df_cat.isna().sum()

fullVisitorId                 0
device_browser                0
device_operatingSystem        0
geoNetwork_city               0
geoNetwork_metro              0
geoNetwork_country            0
geoNetwork_networkDomain      0
geoNetwork_region             0
trafficSource_adContent       0
trafficSource_campaign        0
trafficSource_keyword         0
trafficSource_source          0
trafficSource_referralPath    0
weekday                       0
dtype: int64

In [15]:
train_df_filter.isna().sum() # train_df_filter seems to be the processed data after removing redundant and allegedly less meaningful features

channelGrouping                                 0
date                                            0
fullVisitorId                                   0
visitNumber                                     0
visitStartTime                                  0
totals_bounces                                  0
totals_hits                                     0
totals_newVisits                                0
totals_pageviews                                0
totals_transactionRevenue                       0
device_browser                                  0
device_deviceCategory                           0
device_isMobile                                 0
device_operatingSystem                          0
geoNetwork_city                                 0
geoNetwork_continent                            0
geoNetwork_country                              0
geoNetwork_metro                                0
geoNetwork_networkDomain                        0
geoNetwork_region                               0


In [16]:
#train_df_flat.isna().sum()

channelGrouping                                           0
date                                                      0
fullVisitorId                                             0
sessionId                                                 0
socialEngagementType                                      0
visitId                                                   0
visitNumber                                               0
visitStartTime                                            0
totals_bounces                                       453023
totals_hits                                               0
totals_newVisits                                     200593
totals_pageviews                                        100
totals_transactionRevenue                            892138
totals_visits                                             0
device_browser                                            8
device_browserSize                                   903653
device_browserVersion                   

Note on the values of totals_transactionRevenue: Total transaction revenue is expressed as the value passed to Analytics multiplied by 10^6 (e.g., 2.40 would be given as 2400000).

How many unique visitors did the ecommerce-shop have for the observed period?

In [17]:
unique_customers = train_df_filter.fullVisitorId.nunique()

In [18]:
unique_customers

715129

How many of those generated revenue?

In [19]:
train_df_filter[['fullVisitorId', 'totals_transactionRevenue']].sort_values('totals_transactionRevenue',ascending=False)

,fullVisitorId,totals_transactionRevenue
374904,1957458976293878100,"23,129,500,000.00"
430432,1957458976293878100,"17,855,500,000.00"
380637,5632276788326171571,"16,023,750,000.00"
772974,9417857471295131045,"10,589,140,000.00"
890691,1957458976293878100,"8,677,830,000.00"
...,...,...
302842,6705206951405647737,0.00
302843,262472272057446420,0.00
302844,3627326284569000152,0.00
302845,996291448451917297,0.00


In [20]:
rev_by_customer = train_df_filter[['fullVisitorId', 'totals_transactionRevenue']].groupby('fullVisitorId').sum().sort_values('totals_transactionRevenue',ascending=False)

In [21]:
rev_by_customer_real = rev_by_customer.copy()

In [22]:
rev_by_customer

,totals_transactionRevenue
fullVisitorId,
1957458976293878100,"77,113,430,000.00"
5632276788326171571,"16,023,750,000.00"
9417857471295131045,"15,170,120,000.00"
4471415710206918415,"11,211,100,000.00"
4984366501121503466,"9,513,900,000.00"
...,...
3989336949988348586,0.00
3989359014469271607,0.00
3989364197520680743,0.00


In [23]:
rev_by_customer_real.totals_transactionRevenue = rev_by_customer_real.totals_transactionRevenue/(10**6)

In [24]:
rev_by_customer_real

,totals_transactionRevenue
fullVisitorId,
1957458976293878100,"77,113.43"
5632276788326171571,"16,023.75"
9417857471295131045,"15,170.12"
4471415710206918415,"11,211.10"
4984366501121503466,"9,513.90"
...,...
3989336949988348586,0.00
3989359014469271607,0.00
3989364197520680743,0.00


In [25]:
paying_customers = (rev_by_customer.totals_transactionRevenue > 0).sum()

In [26]:
conversion_rate = paying_customers/unique_customers
conversion_rate

0.013986287788636736

Only 1.4% of all visitors generate (tracked) revenue (assuming missing values mean no revenue)

In [27]:
train_df_filter.head()

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_transactionRevenue,...,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20160902,1131660440785968503,1,1472830385,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
1,Organic Search,20160902,377306020877927890,1,1472880147,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
2,Organic Search,20160902,3895546263509774583,1,1472865386,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
3,Organic Search,20160902,4763447161404445595,1,1472881213,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,google + online,organic,@,google
4,Organic Search,20160902,27294437909732085,2,1472822600,1.00,1,0.00,1.00,0.00,...,@,1,0.00,@,@,1,@,organic,@,google


In [28]:
train_df_filter.nunique()

channelGrouping                                      8
date                                               366
fullVisitorId                                   715129
visitNumber                                        384
visitStartTime                                  886276
totals_bounces                                       2
totals_hits                                        274
totals_newVisits                                     2
totals_pageviews                                   214
totals_transactionRevenue                         5331
device_browser                                      54
device_deviceCategory                                3
device_isMobile                                      2
device_operatingSystem                              20
geoNetwork_city                                    648
geoNetwork_continent                                 6
geoNetwork_country                                 222
geoNetwork_metro                                    93
geoNetwork

We can see that the time period in question are 366 days (not necessarily consecutive ones -- as days in 'dates' where no visits happened would not be recorded)

In [29]:
#train_df_cat.nunique()

fullVisitorId                 714167
device_browser                    54
device_operatingSystem            20
geoNetwork_city                  648
geoNetwork_metro                  93
geoNetwork_country               222
geoNetwork_networkDomain       27520
geoNetwork_region                375
trafficSource_adContent           44
trafficSource_campaign            10
trafficSource_keyword           3507
trafficSource_source             331
trafficSource_referralPath      1260
weekday                            7
dtype: int64

In [30]:
#train_df_filter.columns.isin(train_df_cat.columns)

array([False, False,  True, False, False, False, False, False, False,
       False,  True, False, False,  True,  True, False,  True,  True,
        True,  True, False,  True, False, False, False, False,  True,
       False,  True, False,  True,  True])

In [31]:
#train_df_filter.iloc[:,train_df_filter.columns.isin(train_df_cat.columns)].nunique()

fullVisitorId                 715129
device_browser                    54
device_operatingSystem            20
geoNetwork_city                  648
geoNetwork_country               222
geoNetwork_metro                  93
geoNetwork_networkDomain       28063
geoNetwork_region                375
trafficSource_adContent           45
trafficSource_campaign            10
trafficSource_keyword           3659
trafficSource_referralPath      1475
trafficSource_source             380
dtype: int64

The "filter and "cat columns do not fully correspond. That is, there is more categorical data in "filter than in "cat, probably because some rows have been dropped in "cat due to too much missing data.

" cat also has helpfully a weekday column.

Probably time of the year might also help -- but then these vary with country (etc.) of which there are 222.

In [32]:
#train_df_filter.device_operatingSystem.value_counts()

In [33]:
# pkl = pickle.load('data/data_all_features.pkl')

How many visitors returned?

In [34]:
return_vis = (train_df_filter.fullVisitorId.value_counts() > 1).sum()

In [35]:
return_vis/unique_customers

0.1297542121771037

About 12.5% of all visitors return

In [36]:
customer_rev_visits = rev_by_customer.copy()

In [37]:
#customer_rev_visits['fullVisitorId_sort'] = customer_rev_visits.index.astype(float) # to sort by customerID one has to convert them to floats as int formats are too low precision for such high numbers

In [38]:
customer_rev_visits.sort_values('fullVisitorId', inplace=True)

In [39]:
customer_rev_visits.reset_index(inplace=True)

In [40]:
visits = pd.DataFrame(train_df_filter.fullVisitorId.value_counts())

In [41]:
#visits['fullVisitorId_sort'] = visits.index.astype(float)

In [42]:
visits.columns = ['visits']

In [43]:
visits.reset_index(names=['fullVisitorId', 'visits'], inplace=True)

In [44]:
visits.sort_values('fullVisitorId', inplace=True, ignore_index=True)

In [45]:
visits

,fullVisitorId,visits
0,0000168159078983594,1
1,0000500012543476818,1
2,000084172700693338,1
3,0000945784447887116,1
4,0001213046045394417,1
...,...,...
715124,9999963186378918199,1
715125,999997225970956660,1
715126,999997550040396460,2
715127,9999978264901065827,1


In [46]:
customer_rev_visits['visits'] = visits['visits']

In [47]:
customer_rev_visits

,fullVisitorId,totals_transactionRevenue,visits
0,0000168159078983594,0.00,1
1,0000500012543476818,0.00,1
2,000084172700693338,0.00,1
3,0000945784447887116,0.00,1
4,0001213046045394417,0.00,1
...,...,...,...
715124,9999963186378918199,0.00,1
715125,999997225970956660,0.00,1
715126,999997550040396460,0.00,2
715127,9999978264901065827,0.00,1


In [48]:
paying_customers_visit = customer_rev_visits.sort_values('totals_transactionRevenue', ascending=False).head(paying_customers)

In [49]:
paying_returning = (paying_customers_visit['visits'] > 1).sum() # the majority of paying customers are returning visitors

In [50]:
paying_returning

6831

In [51]:
paying_returning/paying_customers # 65.17%

0.6829634073185363

In [52]:
paying_customers

10002

In [53]:
unique_customers

715129

In [54]:
returning_no_pay = return_vis - paying_returning

In [55]:
no_pay_vis = unique_customers - paying_customers

In [56]:
returning_no_pay/no_pay_vis

0.12190711744125526

Only 11.76% of non-paying visitors return, suggesting that more than one visit (> 1) is a predictor for transactions

Q: Which of these features can be influenced?

channelGrouping                                      8 The Default Channel Group associated with an end user's session for this View.
date                                               366 The date of the session in YYYYMMDD format.
fullVisitorId                                   723854
visitNumber                                        384 The session number for this user. If this is the first session, then this is set to 1.
visitStartTime                                  886276 The timestamp (expressed as POSIX time).
totals_bounces                                       2 Total bounces (for convenience). For a bounced session, the value is 1, otherwise it is null.

Visitors who enter the site and then leave ("bounce") rather than continuing to view other pages within the same site.

totals_hits                                        274 Total number of hits within the session. range: 1-446525

A hit is merely a request for a file from a Web server. A request for a Web page counts as a hit, but so does a request for a graphic on a Web page. Since the number of graphics per page can vary considerably, hits mean very little for comparison purposes.

totals_newVisits                                     2 Total number of new users in session (for convenience). If this is the first visit, this value is 1, otherwise it is null.
totals_pageviews                                   214 Total number of pageviews within the session. -- if it is 1 then bounce is 1 and otherwise 0
totals_transactionRevenue                         5331 Total transaction revenue, expressed as the value passed to Analytics multiplied by 10^6 (e.g., 2.40 would be given as 2400000).
device_browser                                      54 Browser used during visit. Long tail < 10 -- can be summarised as other or so
device_deviceCategory                                3 desktop/mobile/tablet -- some redundancy with browser
device_isMobile                                      2 0/1 - some redundancy with deviceCategory
device_operatingSystem                              20
geoNetwork_city                                    648
geoNetwork_continent                                 6
geoNetwork_country                                 222 The country from which sessions originated, based on IP address.
geoNetwork_metro                                    93 The Designated Market Area (DMA) from which sessions originate.
geoNetwork_networkDomain                         28063 The domain name of user's internet service proviser (ISP), derived from the domain name registered to the ISP's IP address.
geoNetwork_region                                  375
geoNetwork_subContinent                             23
trafficSource_adContent                             45 The ad content of the traffic source. Can be set by the utm_content URL parameter.
trafficSource_adwordsClickInfo.adNetworkType         3 Network Type. Takes one of the following values: {“Google Search", "Content", "Search partners", "Ad Exchange", "Yahoo Japan Search", "Yahoo Japan AFS", “unknown”} -- most are "@", prbbly missing val
trafficSource_adwordsClickInfo.isVideoAd             1 True if it is a Trueview video ad. -- has just one value and is therefore WITHOUT MEANING
trafficSource_adwordsClickInfo.page                  9 Page number in search results where the ad was shown. -- most have missing values? 0? Or is this the first page
trafficSource_adwordsClickInfo.slot                  3 Position of the Ad. Takes one of the following values:{“RHS", "Top"} -- @ is missing data, most are @
trafficSource_campaign                              10 The campaign value. Usually set by the utm_campaign URL parameter. -- most data is missing
trafficSource_isTrueDirect                           2 True if the source of the session was Direct (meaning the user typed the name of your website URL into the browser or came to your site via a bookmark), This field will also be true if 2 successive but distinct sessions have exactly the same campaign details. Otherwise NULL.
trafficSource_keyword                             3659 The keyword of the traffic source, usually set when the trafficSource.medium is "organic" or "cpc". Can be set by the utm_term URL parameter.
trafficSource_medium                                 6 The medium of the traffic source. Could be "organic", "cpc", "referral", or the value of the utm_medium URL parameter.   
trafficSource_referralPath                        1475 If trafficSource.medium is "referral", then this is set to the path of the referrer. (The host name of the referrer is in trafficSource.source.)
trafficSource_source                               380 The source of the traffic source. Could be the name of the search engine, the referring hostname, or a value of the utm_source URL parameter.

In [57]:
train_df_filter['visitNumber'].value_counts()

1      702435
2       92407
3       35799
4       19132
5       11595
        ...  
364         1
363         1
370         1
307         1
387         1
Name: visitNumber, Length: 384, dtype: int64

In [58]:
train_df_filter.describe()

,date,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_transactionRevenue,device_isMobile,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_isTrueDirect
count,"902,755.00","902,755.00","902,755.00","902,755.00","902,755.00","902,755.00","902,755.00","902,755.00","902,755.00","902,755.00","902,755.00","902,755.00"
mean,"20,165,884.79",2.27,"1,485,006,191.83",0.50,4.59,0.78,3.84,"1,705,620.47",0.26,1.00,0.02,0.30
std,"4,697.68",9.29,"9,021,360.51",0.50,9.63,0.42,7.02,"52,804,453.36",0.44,0.00,0.16,0.46
min,"20,160,801.00",1.00,"1,470,034,812.00",0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
25%,"20,161,027.00",1.00,"1,477,561,509.00",0.00,1.00,1.00,1.00,0.00,0.00,1.00,0.00,0.00
50%,"20,170,108.00",1.00,"1,483,945,338.00",0.00,2.00,1.00,1.00,0.00,0.00,1.00,0.00,0.00
75%,"20,170,420.00",1.00,"1,492,755,024.50",1.00,4.00,1.00,4.00,0.00,1.00,1.00,0.00,1.00
max,"20,170,801.00",395.00,"1,501,657,193.00",1.00,500.00,1.00,469.00,"23,129,500,000.00",1.00,1.00,14.00,1.00


In [59]:
#train_df_flat.nunique()

channelGrouping                                           8
date                                                    366
fullVisitorId                                        715119
sessionId                                            902755
socialEngagementType                                      1
visitId                                              886303
visitNumber                                             384
visitStartTime                                       887159
totals_bounces                                            1
totals_hits                                             274
totals_newVisits                                          1
totals_pageviews                                        213
totals_transactionRevenue                              5332
totals_visits                                             1
device_browser                                           53
device_browserSize                                        0
device_browserVersion                   

EDA shows that there is very little correlation between revenue and other variables. This is possibly due to the difference in scale -- revenue is in millions, features are mostly categorical. Let's change the revenue feature.

Looks like most features can be discarded.

Also, data is not aggregated by customer but is session based. This allows, however, to identify features that play a role from session to session in generating revenue.

There also seem to be some duplicates that need to be removed. After checking this, there seem to be no duplicates -- probably the impression comes from the fact that the profiling cannot deal with visitorIds.

In [60]:
train_df_filter['revenue_generated'] = (train_df_filter['totals_transactionRevenue'] > 0).astype(int)

In [61]:
train_df_filter['revenue_amount'] = train_df_filter['totals_transactionRevenue']/(10**6)

In [62]:
train_df_filter.drop(columns='totals_transactionRevenue', inplace=True)

In [63]:
train_df_filter.head()

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,device_browser,...,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,revenue_generated,revenue_amount
0,Organic Search,20160902,1131660440785968503,1,1472830385,1.00,1,1.00,1.00,Chrome,...,0.00,@,@,0,@,organic,@,google,0,0.00
1,Organic Search,20160902,377306020877927890,1,1472880147,1.00,1,1.00,1.00,Firefox,...,0.00,@,@,0,@,organic,@,google,0,0.00
2,Organic Search,20160902,3895546263509774583,1,1472865386,1.00,1,1.00,1.00,Chrome,...,0.00,@,@,0,@,organic,@,google,0,0.00
3,Organic Search,20160902,4763447161404445595,1,1472881213,1.00,1,1.00,1.00,UC Browser,...,0.00,@,@,0,google + online,organic,@,google,0,0.00
4,Organic Search,20160902,27294437909732085,2,1472822600,1.00,1,0.00,1.00,Chrome,...,0.00,@,@,1,@,organic,@,google,0,0.00


In [64]:
# eda = train_df_class.profile_report()
# eda

In [65]:
#correlations = eda.description_set["correlations"]

In [66]:
#correlations['auto']

In [67]:
train_df_filter.drop(columns='trafficSource_adwordsClickInfo.isVideoAd', inplace=True) ## ADD TO PREPROCESSING. Feature has a constant value and has thus no value

In [68]:
train_df_filter.drop_duplicates(ignore_index=True).shape ## ADD TO PREPROCESSING -- there are no duplicates

(902755, 32)

In [69]:
#eda1 = train_df_filter.drop(columns='fullVisitorId').profile_report()
#eda1

Now, taking IDs into account, there are no duplicates.

How do the values change for a customer over visits?

In [70]:
train_df_filter[train_df_filter['fullVisitorId']=='4471415710206918415'] 

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,device_browser,...,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,revenue_generated,revenue_amount
31888,Referral,20161123,4471415710206918415,30,1479941178,0.00,2,0.00,2.00,Chrome,...,0.00,@,@,1,@,referral,@,mall.googleplex.com,0,0.00
97032,Referral,20160816,4471415710206918415,13,1471365716,1.00,1,0.00,1.00,Chrome,...,0.00,@,@,0,@,referral,/_/scs/mail-static/_/js/k=gmail.main.en.sXDiEp...,mail.google.com,0,0.00
138506,Referral,20170320,4471415710206918415,36,1490042485,0.00,13,0.00,9.00,Chrome,...,0.00,@,@,1,@,referral,@,mall.googleplex.com,0,0.00
141819,Organic Search,20170410,4471415710206918415,41,1491855205,0.00,5,0.00,4.00,Chrome,...,0.00,@,@,1,@,organic,@,google,0,0.00
151690,Referral,20170329,4471415710206918415,39,1490852758,0.00,2,0.00,2.00,Chrome,...,0.00,@,@,1,@,referral,@,mall.googleplex.com,0,0.00
158549,Referral,20170308,4471415710206918415,34,1489014943,0.00,2,0.00,2.00,Chrome,...,0.00,@,@,1,@,referral,@,mall.googleplex.com,0,0.00
199306,Referral,20160915,4471415710206918415,19,1473979062,0.00,14,0.00,10.00,Chrome,...,0.00,@,@,1,@,referral,/_/scs/mail-static/_/js/k=gmail.main.en.sXDiEp...,mail.google.com,0,0.00
305027,Referral,20160913,4471415710206918415,18,1473788517,0.00,2,0.00,2.00,Chrome,...,0.00,@,@,1,@,referral,/_/scs/mail-static/_/js/k=gmail.main.en.sXDiEp...,mail.google.com,0,0.00
322581,Referral,20160826,4471415710206918415,16,1472263330,0.00,3,0.00,3.00,Chrome,...,0.00,@,@,1,@,referral,/_/scs/mail-static/_/js/k=gmail.main.en.sXDiEp...,mail.google.com,0,0.00
322680,Referral,20160826,4471415710206918415,15,1472246417,0.00,8,0.00,6.00,Chrome,...,0.00,@,@,1,@,referral,/_/scs/mail-static/_/js/k=gmail.main.en.sXDiEp...,mail.google.com,0,0.00


For this customer, for instance, most visits were without transactions and were referrals from google mails or mall.googleplex

In [71]:
#train_df_filter['fullVisitorId'].astype(str).sort_values() ## some of the ids were previously strings and some were ints, this is why sort did not work, converting everything to strings solves the sorting problem

__DONE__ __To Do__: add Id-> string to initial preprocessing

In [72]:
#eda1.to_file('/home/slawa/code/code-rep0/projects/GA_revenue_analytics/eda_train_df_filter.html')

In [73]:
train_df_filter[train_df_filter['totals_pageviews']==0]

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,device_browser,...,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,revenue_generated,revenue_amount
27193,Social,20161112,1854440936315683918,4,1478940943,0.00,1,0.00,0.00,Chrome,...,0.00,@,@,1,@,referral,/yt/about/es/,youtube.com,0,0.00
27228,Social,20161112,1854440936315683918,7,1478952119,0.00,1,0.00,0.00,Chrome,...,0.00,@,@,1,@,referral,/yt/about/es/,youtube.com,0,0.00
35456,Direct,20170501,4031993507382416569,3,1493666949,0.00,1,0.00,0.00,Chrome,...,0.00,@,@,1,@,@,@,(direct),0,0.00
68959,Direct,20161209,1566880879657202729,2,1481328178,0.00,2,0.00,0.00,Chrome,...,0.00,@,@,1,@,@,@,(direct),0,0.00
86474,Organic Search,20161129,7865620284367959676,4,1480441847,0.00,4,0.00,0.00,Chrome,...,0.00,@,@,1,@,organic,@,google,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854271,Direct,20160818,2126636044556374062,20,1471532792,0.00,1,0.00,0.00,Chrome,...,0.00,@,@,1,@,@,@,(direct),0,0.00
856122,Direct,20170520,8717262821534000131,3,1495335373,0.00,1,0.00,0.00,Safari,...,0.00,@,@,1,@,@,@,(direct),0,0.00
881304,Referral,20161019,5617770153245303309,4,1476907903,0.00,2,0.00,0.00,Chrome,...,0.00,@,@,1,@,referral,@,mall.googleplex.com,0,0.00
883263,Paid Search,20170319,7417510682931989623,3,1489918339,0.00,1,0.00,0.00,Firefox,...,0.00,@,@,1,(Remarketing/Content targeting),cpc,@,google,0,0.00


In [74]:
train_df_filter['trafficSource_keyword'].value_counts().head(20) #--> can be dropped and has high cardinality

@                                  868435
6qEhsCssdK0z36ri                    11486
(Remarketing/Content targeting)      2295
1hZbAqLCbjwfgOH7                     2260
google merchandise store             2205
Google Merchandise                   1642
google store                         1277
youtube                               568
(User vertical targeting)             489
1X4Me6ZKNV0zg-jV                      467
(automatic matching)                  427
+google +store                        377
+Google +Merchandise                  351
(content targeting)                   299
+google +merchandise +store           258
YouTube Merchandise                   254
youtube merch                         247
google merchandise                    238
google merch                          209
Youtube                               166
Name: trafficSource_keyword, dtype: int64

In [75]:
train_df_filter['geoNetwork_networkDomain'].value_counts().head(20) #--> high cardinality, can be dropped

@                        390535
comcast.net               28687
rr.com                    14802
verizon.net               13630
ttnet.com.tr              13224
comcastbusiness.net        9975
hinet.net                  7902
virginm.net                6411
3bb.co.th                  6042
prod-infinitum.com.mx      5957
cox.net                    5804
sbcglobal.net              5378
btcentralplus.com          5302
att.net                    5226
google.com                 5035
optonline.net              4970
totbb.net                  4892
vnpt.vn                    4507
asianet.co.th              4370
pldt.net                   4007
Name: geoNetwork_networkDomain, dtype: int64

In [76]:
#train_df_filter['fullVisitorId', 'country'].groupby('country')

In [77]:
#visitor_country = train_df_filter[['fullVisitorId', 'geoNetwork_country','revenue_generated']].groupby('fullVisitorId').agg(country=('geoNetwork_country', 'unique'), 
                                                                                                                                 #paying_visitors=('revenue_generated', 'sum'))
#.groupby('geoNetwork_country').agg(unique_visitors=('fullVisitorId', 'count'), 
#                                                                                                                                 paying_visitors=('revenue_generated', 'sum'))

IndentationError: unexpected indent (2308152708.py, line 2)

Revenue per customer per country

__To Do__: would also be interesting to see amount spent

In [82]:
# visitor_country = train_df_filter[['fullVisitorId', 'geoNetwork_country','revenue_generated']].groupby('fullVisitorId').agg(country=('geoNetwork_country', pd.Series.mode), 
#                                                                                                                                  paying_visitors=('revenue_generated', 'sum'))

# visitor_country.reset_index(inplace=True)

# visitor_country

# visitor_country['country'] = visitor_country['country'].astype(str) 
# # that was the crucial step again, as country was in some cases where several countries were concerned an array. How is this possible with mode? Because several entries were equally frequent

In [83]:
# visitor_country

,fullVisitorId,country,paying_visitors
0,0000168159078983594,Taiwan,0
1,0000500012543476818,United States,0
2,000084172700693338,South Korea,0
3,0000945784447887116,United States,0
4,0001213046045394417,United States,0
...,...,...,...
715124,9999963186378918199,United Kingdom,0
715125,999997225970956660,United States,0
715126,999997550040396460,United States,0
715127,9999978264901065827,United States,0


In [88]:
#path = !pwd

In [91]:
#visitor_country.to_csv(f'{path[0]}/data/visitor_country.csv')

In [92]:
# visitors_by_country = visitor_country.groupby('country').agg(unique_visitors=('fullVisitorId', 'count'), 
#                                        paying_visitors=('paying_visitors', 'sum'))

# visitors_by_country['conversion'] = (visitors_by_country['paying_visitors']/visitors_by_country['unique_visitors'])*100

In [95]:
#visitors_by_country.to_csv(f'{path[0]}/data/visitors_by_country.csv')

In [97]:
visitor_country = pd.read_csv('data/visitor_country.csv')
visitor_country['country'] = visitor_country['country'].astype(str) 

In [98]:
visitors_by_country = pd.read_csv('data/visitors_by_country.csv') # no string conversion of country necessary?

In [99]:
visitors_by_country[visitors_by_country['paying_visitors']>0].sort_values('conversion',ascending=False).head(50)

,country,unique_visitors,paying_visitors,conversion
489,['Switzerland' 'United Kingdom' 'United States'],1,1,100.00
371,['Guadeloupe' 'United States'],1,1,100.00
7,Anguilla,1,1,100.00
226,['@' 'Indonesia'],3,2,66.67
504,['United States' 'Venezuela'],10,5,50.00
433,['Netherlands' 'Russia'],2,1,50.00
450,['Philippines' 'United States'],3,1,33.33
498,['Ukraine' 'United States'],4,1,25.00
337,['France' 'Germany'],4,1,25.00
471,['Singapore' 'United States'],16,4,25.00


Revenue per visit by country

In [ ]:
visit_rev_country = train_df_filter[['fullVisitorId', 'geoNetwork_country','revenue_generated']].groupby('geoNetwork_country').agg(unique_visitors=('fullVisitorId', 'count'), 
                                                                                                                                 paying_visitors=('revenue_generated', 'sum'))

In [ ]:
visit_rev_country

In [ ]:
paying_visitors_country = visit_rev_country[visit_rev_country['paying_visitors']>0]

In [ ]:
paying_visitors_country['conversion'] = (visit_rev_country['paying_visitors']/visit_rev_country['unique_visitors'])*100

In [ ]:
paying_visitors_country.sort_values('conversion', ascending=False).head(15)

In [ ]:
visit_rev_country['unique_visitors'].sum()

How much time passes between visits?

In [ ]:
import datetime

In [107]:
train_df_filter.date = pd.to_datetime(train_df_filter.date, format='%Y%m%d')

In [ ]:
train_df_filter.date[102323] - train_df_filter.date[0]

In [ ]:
visits_1 = train_df_filter[train_df_filter['fullVisitorId']=='4471415710206918415'].sort_values('date').reset_index() ## one could also sort by visit_numbers

In [ ]:
visits_1.date[2]-visits_1.date[1]

In [ ]:
days_passed = []

for i in range(len(visits_1.date)-1):
    days_passed.append((visits_1.date[i+1]-visits_1.date[i]).days)



In [208]:
train_df_filter['days_since_visit'] = np.nan

In [209]:
train_df_filter.head()

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,device_browser,...,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,revenue_generated,revenue_amount,days_since_visit
0,Organic Search,2016-09-02,1131660440785968503,1,1472830385,1.00,1,1.00,1.00,Chrome,...,@,@,0,@,organic,@,google,0,0.00,None
1,Organic Search,2016-09-02,377306020877927890,1,1472880147,1.00,1,1.00,1.00,Firefox,...,@,@,0,@,organic,@,google,0,0.00,None
2,Organic Search,2016-09-02,3895546263509774583,1,1472865386,1.00,1,1.00,1.00,Chrome,...,@,@,0,@,organic,@,google,0,0.00,None
3,Organic Search,2016-09-02,4763447161404445595,1,1472881213,1.00,1,1.00,1.00,UC Browser,...,@,@,0,google + online,organic,@,google,0,0.00,None
4,Organic Search,2016-09-02,27294437909732085,2,1472822600,1.00,1,0.00,1.00,Chrome,...,@,@,1,@,organic,@,google,0,0.00,None


Some visit numbers are missing

In [100]:
visits_per_visitor = train_df_filter['fullVisitorId'].value_counts() 

In [101]:
visits_per_visitor = visits_per_visitor.reset_index()

In [102]:
visits_per_visitor.columns = ['fullVisitorId', 'visits']

In [103]:
returning_visitors = visits_per_visitor[visits_per_visitor['visits']>1]

In [104]:
returning_visitors

,fullVisitorId,visits
0,1957458976293878100,278
1,824839726118485274,242
2,3608475193341679870,200
3,1856749147915772585,199
4,3269834865385146569,155
...,...,...
92786,628300805200742094,2
92787,3389168322304694026,2
92788,1072729019643723405,2
92789,5644012794353911600,2


In [108]:
train_df_filter.head()

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,device_browser,...,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,revenue_generated,revenue_amount,days_since_visit
0,Organic Search,2016-09-02,1131660440785968503,1,1472830385,1.00,1,1.00,1.00,Chrome,...,@,@,0,@,organic,@,google,0,0.00,NaN
1,Organic Search,2016-09-02,377306020877927890,1,1472880147,1.00,1,1.00,1.00,Firefox,...,@,@,0,@,organic,@,google,0,0.00,NaN
2,Organic Search,2016-09-02,3895546263509774583,1,1472865386,1.00,1,1.00,1.00,Chrome,...,@,@,0,@,organic,@,google,0,0.00,NaN
3,Organic Search,2016-09-02,4763447161404445595,1,1472881213,1.00,1,1.00,1.00,UC Browser,...,@,@,0,google + online,organic,@,google,0,0.00,NaN
4,Organic Search,2016-09-02,27294437909732085,2,1472822600,1.00,1,0.00,1.00,Chrome,...,@,@,1,@,organic,@,google,0,0.00,NaN


In [210]:
visit_dates = train_df_filter[['fullVisitorId', 'date', 'days_since_visit']]

In [212]:
visit_dates = visit_dates.sort_values(['fullVisitorId', 'date'])

In [220]:
visit_dates[['last_visit_id', 'last_visit_date']] = visit_dates.shift(+1)[['fullVisitorId', 'date']]

In [225]:
def calc_days_since_visit(row):
    if row['fullVisitorId'] == row['last_visit_id']:
        return (row['date'] - row['last_visit_date']).days
    else:
        return np.nan

visit_dates['days_since_visit'] = visit_dates.apply(calc_days_since_visit, axis=1)

In [231]:
visit_dates[visit_dates.days_since_visit.notna()]

,fullVisitorId,date,days_since_visit,last_visit_id,last_visit_date
39835,0006191747463833846,2017-06-30,0.00,0006191747463833846,2017-06-30
37859,0008708493573668829,2017-06-11,0.00,0008708493573668829,2017-06-11
503217,0021430049270067403,2016-12-22,9.00,0021430049270067403,2016-12-13
495717,0021728288823925547,2016-09-28,0.00,0021728288823925547,2016-09-28
425064,0025489527250127533,2016-08-14,0.00,0025489527250127533,2016-08-14
...,...,...,...,...,...
447552,9999520059803839410,2017-05-04,9.00,9999520059803839410,2017-04-25
529871,9999594748540438502,2017-07-05,32.00,9999594748540438502,2017-06-03
627265,9999632697226950818,2016-12-12,0.00,9999632697226950818,2016-12-12
629017,9999761280717362235,2016-12-12,75.00,9999761280717362235,2016-09-28


In [232]:
visit_dates.sort_index(inplace=True)

In [234]:
train_df_filter.days_since_visit = visit_dates.days_since_visit

In [235]:
train_df_filter

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,device_browser,...,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,revenue_generated,revenue_amount,days_since_visit
0,Organic Search,2016-09-02,1131660440785968503,1,1472830385,1.00,1,1.00,1.00,Chrome,...,@,@,0,@,organic,@,google,0,0.00,NaN
1,Organic Search,2016-09-02,377306020877927890,1,1472880147,1.00,1,1.00,1.00,Firefox,...,@,@,0,@,organic,@,google,0,0.00,NaN
2,Organic Search,2016-09-02,3895546263509774583,1,1472865386,1.00,1,1.00,1.00,Chrome,...,@,@,0,@,organic,@,google,0,0.00,NaN
3,Organic Search,2016-09-02,4763447161404445595,1,1472881213,1.00,1,1.00,1.00,UC Browser,...,@,@,0,google + online,organic,@,google,0,0.00,NaN
4,Organic Search,2016-09-02,27294437909732085,2,1472822600,1.00,1,0.00,1.00,Chrome,...,@,@,1,@,organic,@,google,0,0.00,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902750,Social,2017-01-04,5123779100307500332,1,1483554750,0.00,17,1.00,15.00,Chrome,...,@,@,0,@,referral,/yt/about/,youtube.com,0,0.00,NaN
902751,Social,2017-01-04,7231728964973959842,1,1483543798,0.00,18,1.00,13.00,Chrome,...,@,@,0,@,referral,/yt/about/,youtube.com,0,0.00,NaN
902752,Social,2017-01-04,5744576632396406899,1,1483526434,0.00,24,1.00,21.00,Android Webview,...,@,@,0,@,referral,/yt/about/ko/,youtube.com,0,0.00,NaN
902753,Social,2017-01-04,2709355455991750775,1,1483592864,0.00,24,1.00,22.00,Chrome,...,@,@,0,@,referral,/l.php,facebook.com,0,0.00,0.00


In [318]:
temp = train_df_filter[train_df_filter['fullVisitorId']=='4471415710206918415'][['date', 'days_since_visit']].sort_values('date').reset_index()

In [325]:
temp.sort_values('index').reset_index()['days_since_visit']

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
30   NaN
31   NaN
32   NaN
33   NaN
Name: days_since_visit, dtype: float64